# FLOW_API Demonstration

## FLOW project meeting

### Leuven, 10-11 Sep 2024

In [ ]:
%matplotlib inline
import os
import xarray as xr
import matplotlib.pyplot as plt
from ncplot import view

from windIO.utils.yml_utils import validate_yaml
from windIO.utils import plant_schemas_path

from flow_api import run_foxes, run_pywake #, run_wayve, run_code_saturne

## Currently available example cases

In [ ]:
!(cd ../examples/cases && tree -L 1)

## Example 1: Four turbines in a row, homogeneous inflow timeseries

This case is called "windio_4turbines". The input data file structure looks like this:

In [ ]:
!(cd ../examples/cases/windio_4turbines && tree)

The main file of this case is called "FLOW_toy_study_wind_energy_system.yaml". It follows the windio schema. Notice how other files are referenced via the `!include` command:

In [ ]:
!cat ../examples/cases/windio_4turbines/wind_energy_system/system.yaml

This is the "analysis" input file:

In [ ]:
!head -n 27 ../examples/cases/windio_4turbines/wind_energy_system/analysis.yaml

We can validate the input with windIO's `validate` function:

In [ ]:
input_yaml = "../examples/cases/windio_4turbines/wind_energy_system/system.yaml"
validate_yaml(input_yaml, plant_schemas_path + 'wind_energy_system.yaml')

The `flow_model` under `attributes` states `foxes`, which means that a call of the main `flow_api` would ask `foxes` to compute results. However, this choice can be overruled by specified functions/commands.

**Provided python functions:**
- *run_api(input_yaml)*: Run the case with the flow model specified in the yaml file
- *run_foxes(input_yaml)*: Run the case with `foxes`
- *run_pywake(input_yaml)*: Run the case with `PyWake`
- *run_wayve(input_yaml)*: Run the case with `WAYVE`
- *run_code_saturne(input_yaml)*: Run the case with `code_saturne`

**Provided command-line tools:**
- *flow_api input_yaml*: Run the case with the flow model specified in the yaml file
- *flow_api_foxes input_yaml)*: Run the case with `foxes`
- *flow_api_pywake input_yaml)*: Run the case with `PyWake`
- *flow_api_wayve input_yaml)*: Run the case with `WAYVE`
- *flow_api_code_saturne input_yaml)*: Run the case with `code_saturne`

For example, we can run the main API call like as follows (Note that the "!" is only needed because we are in a notebook here, not a terminal):

In [ ]:
!flow_api ../examples/cases/windio_4turbines/wind_energy_system/system.yaml

Notice that the outputs that were defined in the `yaml` above triggered the writing of two output files:

In [ ]:
!tree results

In [ ]:
view("results/flow_field.nc", vars= "wind_speed")

In [ ]:
view("results/turbine_data.nc")

## Comparison of foxes and PyWake

Let's re-run `foxes`, but now a version that does not export the flow field, and now using the Python function:

In [ ]:
input_yaml = "../examples/cases/windio_4turbines/wind_energy_system/system_no_field.yaml"
validate_yaml(input_yaml, plant_schemas_path + 'wind_energy_system.yaml')

In [ ]:
!rm -rf results
run_foxes(input_yaml)

In [ ]:
foxes_dat = xr.load_dataset('./results/turbine_data.nc')
foxes_dat 

We now run `PyWake` on the same input, by using the `run_pywake` function. It writes to a folder called `output`:

In [ ]:
!rm -rf output && mkdir output
run_pywake(input_yaml)

In [ ]:
pywake_dat = xr.load_dataset('output/PowerTable.nc')
pywake_dat

Now that all results have been computer - let's compare the outcome:

In [ ]:
plt.plot(pywake_dat.turbine, pywake_dat.power.mean('time'),c='orange', label="pywake")
plt.plot(foxes_dat.turbine, foxes_dat.power.mean('time'), ls='-.',c='blue', label="foxes")
plt.xlabel('Turbine Index')
plt.ylabel('Mean power [W]')
plt.legend()
plt.show()

In [ ]:
plt.scatter(pywake_dat.wd, pywake_dat.sel(turbine=1).power, c='orange', marker='o',alpha=0.8, label="pywake")
plt.scatter(pywake_dat.wd, foxes_dat.sel(turbine=1).power, c='blue', marker='x',alpha=0.5, label="foxes")
plt.xlabel('Wind direction [°]')
plt.ylabel('Mean power [W]')
plt.legend()
plt.show()

## Example 2: Four turbines, vertical wind inflow profile

We now want to look at the effect of averaging a background wind profile over the rotor disc area. This is the inflow data, applied to the same four-turbine wind farm of above:

In [ ]:
view("../examples/cases/windio_4turbines_profiles_stable/plant_energy_resource/Stochastic_nieuwstadt_profiles.nc")

First we wish to compute results by evaluating the background at the rotor centre points only. The wakes are evaluated on a weighted 7 x 7 grid of points. These are the corresponding windio yaml files:

In [ ]:
input_yaml_centre = "../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/system.yaml"
input_yaml_grid = "../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/system_grid.yaml"
validate_yaml(input_yaml_centre, plant_schemas_path + 'wind_energy_system.yaml')
validate_yaml(input_yaml_grid, plant_schemas_path + 'wind_energy_system.yaml')

In [ ]:
!head -n 27 ../examples/cases/windio_4turbines_profiles_stable/wind_energy_system/analysis_grid.yaml

In [ ]:
!rm -rf results
run_foxes(input_yaml_centre)
centre_dat = xr.load_dataset('./results/turbine_data.nc')

Now we re-run this, using the 7 x 7 grid also for the background averagine. These are the input yaml files, notice the difference in the rotor averaging:

In [ ]:
!rm -rf results/turbine_data_grid.nc
run_foxes(input_yaml_grid)
grid_dat = xr.load_dataset('./results/turbine_data_grid.nc')

This shows a comparison of the time-averaged power results of the four turbines:

In [ ]:
P_centre = centre_dat.power.mean("time")
P_grid = grid_dat.power.mean("time")
plt.plot(centre_dat.turbine, P_centre, label="centre")
plt.plot(grid_dat.turbine, P_grid, label="grid49")
plt.legend()
plt.xlabel("Turbine index")
plt.ylabel("Mean power [W]")
plt.show()